# Basic face recognizer using pre trained model

Below the is the implementation of a very basic face recognizer which can identify the face of the person showing on a web cam.
The implementation is inspired by two path breaking papers on facial recognition using deep convoluted neural network, namely FaceNet and DeepFace.

I have used pre trained model Keras-OpenFace which is an open source Keras implementation of the OpenFace (Originally Torch implemented) 

The pretrained model that I have used is by Victor Sy Wang's implementation and was loaded using his code: https://github.com/iwantooxxoox/Keras-OpenFace.

# Import the prerequisite libraries

We will be importing utils.py from https://github.com/iwantooxxoox/Keras-OpenFace/blob/master/utils.py (available with code) which contains utility functions to create the neural network and load the weights assoiated with it.

In [21]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utils import LRN2D
import utils

%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Contructing the neural network model
The model here constructed is based on FaceNet's Inception model.

The implementation of model is available at: https://github.com/iwantooxxoox/Keras-OpenFace

In [22]:
myInput = Input(shape=(96, 96, 3))

x = ZeroPadding2D(padding=(3, 3), input_shape=(96, 96, 3))(myInput)
x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn1')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)
x = Lambda(LRN2D, name='lrn_1')(x)
x = Conv2D(64, (1, 1), name='conv2')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn2')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = Conv2D(192, (3, 3), name='conv3')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn3')(x)
x = Activation('relu')(x)
x = Lambda(LRN2D, name='lrn_2')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

# Inception3a
inception_3a_3x3 = Conv2D(96, (1, 1), name='inception_3a_3x3_conv1')(x)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn1')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)
inception_3a_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3)
inception_3a_3x3 = Conv2D(128, (3, 3), name='inception_3a_3x3_conv2')(inception_3a_3x3)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn2')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)

inception_3a_5x5 = Conv2D(16, (1, 1), name='inception_3a_5x5_conv1')(x)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn1')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)
inception_3a_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5)
inception_3a_5x5 = Conv2D(32, (5, 5), name='inception_3a_5x5_conv2')(inception_3a_5x5)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn2')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)

inception_3a_pool = MaxPooling2D(pool_size=3, strides=2)(x)
inception_3a_pool = Conv2D(32, (1, 1), name='inception_3a_pool_conv')(inception_3a_pool)
inception_3a_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_pool_bn')(inception_3a_pool)
inception_3a_pool = Activation('relu')(inception_3a_pool)
inception_3a_pool = ZeroPadding2D(padding=((3, 4), (3, 4)))(inception_3a_pool)

inception_3a_1x1 = Conv2D(64, (1, 1), name='inception_3a_1x1_conv')(x)
inception_3a_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_1x1_bn')(inception_3a_1x1)
inception_3a_1x1 = Activation('relu')(inception_3a_1x1)

inception_3a = concatenate([inception_3a_3x3, inception_3a_5x5, inception_3a_pool, inception_3a_1x1], axis=3)

# Inception3b
inception_3b_3x3 = Conv2D(96, (1, 1), name='inception_3b_3x3_conv1')(inception_3a)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn1')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)
inception_3b_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3)
inception_3b_3x3 = Conv2D(128, (3, 3), name='inception_3b_3x3_conv2')(inception_3b_3x3)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn2')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)

inception_3b_5x5 = Conv2D(32, (1, 1), name='inception_3b_5x5_conv1')(inception_3a)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn1')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)
inception_3b_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5)
inception_3b_5x5 = Conv2D(64, (5, 5), name='inception_3b_5x5_conv2')(inception_3b_5x5)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn2')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)

inception_3b_pool = Lambda(lambda x: x**2, name='power2_3b')(inception_3a)
inception_3b_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: x*9, name='mult9_3b')(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_3b')(inception_3b_pool)
inception_3b_pool = Conv2D(64, (1, 1), name='inception_3b_pool_conv')(inception_3b_pool)
inception_3b_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_pool_bn')(inception_3b_pool)
inception_3b_pool = Activation('relu')(inception_3b_pool)
inception_3b_pool = ZeroPadding2D(padding=(4, 4))(inception_3b_pool)

inception_3b_1x1 = Conv2D(64, (1, 1), name='inception_3b_1x1_conv')(inception_3a)
inception_3b_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_1x1_bn')(inception_3b_1x1)
inception_3b_1x1 = Activation('relu')(inception_3b_1x1)

inception_3b = concatenate([inception_3b_3x3, inception_3b_5x5, inception_3b_pool, inception_3b_1x1], axis=3)

# Inception3c
inception_3c_3x3 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_3x3',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))

inception_3c_5x5 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))

inception_3c_pool = MaxPooling2D(pool_size=3, strides=2)(inception_3b)
inception_3c_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_3c_pool)

inception_3c = concatenate([inception_3c_3x3, inception_3c_5x5, inception_3c_pool], axis=3)

#inception 4a
inception_4a_3x3 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=192,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_4a_5x5 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(1, 1),
                                   padding=(2, 2))

inception_4a_pool = Lambda(lambda x: x**2, name='power2_4a')(inception_3c)
inception_4a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: x*9, name='mult9_4a')(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_4a')(inception_4a_pool)
inception_4a_pool = utils.conv2d_bn(inception_4a_pool,
                                   layer='inception_4a_pool',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   padding=(2, 2))
inception_4a_1x1 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_4a = concatenate([inception_4a_3x3, inception_4a_5x5, inception_4a_pool, inception_4a_1x1], axis=3)

#inception4e
inception_4e_3x3 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_3x3',
                                   cv1_out=160,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))
inception_4e_5x5 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_5x5',
                                   cv1_out=64,
                                   cv1_filter=(1, 1),
                                   cv2_out=128,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))
inception_4e_pool = MaxPooling2D(pool_size=3, strides=2)(inception_4a)
inception_4e_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_4e_pool)

inception_4e = concatenate([inception_4e_3x3, inception_4e_5x5, inception_4e_pool], axis=3)

#inception5a
inception_5a_3x3 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))

inception_5a_pool = Lambda(lambda x: x**2, name='power2_5a')(inception_4e)
inception_5a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: x*9, name='mult9_5a')(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_5a')(inception_5a_pool)
inception_5a_pool = utils.conv2d_bn(inception_5a_pool,
                                   layer='inception_5a_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   padding=(1, 1))
inception_5a_1x1 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))

inception_5a = concatenate([inception_5a_3x3, inception_5a_pool, inception_5a_1x1], axis=3)

#inception_5b
inception_5b_3x3 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_5b_pool = MaxPooling2D(pool_size=3, strides=2)(inception_5a)
inception_5b_pool = utils.conv2d_bn(inception_5b_pool,
                                   layer='inception_5b_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1))
inception_5b_pool = ZeroPadding2D(padding=(1, 1))(inception_5b_pool)

inception_5b_1x1 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_5b = concatenate([inception_5b_3x3, inception_5b_pool, inception_5b_1x1], axis=3)

av_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(inception_5b)
reshape_layer = Flatten()(av_pool)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)


# Final Model
model = Model(inputs=[myInput], outputs=norm_layer)

# Loading the model with pretrained weights

FaceNet is trained by minimizing the triplet loss. I will be  loading a previously trained model. weights are avaiable at https://github.com/iwantooxxoox/Keras-OpenFace in the "weights" folder which is also provided in this source.

This can take a couple of minutes to execute and depends on the speed of your machine.


In [23]:
# Load weights from csv files (which was exported from Openface torch model)
weights = utils.weights
weights_dict = utils.load_weights()

# Set layer weights of the model
for name in weights:
  if model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])
  elif model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])

## About <font color=blue>image_to_embedding</font> function        
When the model is loaded with pre trained weights, then we can create the **128 dimensional embedding vectors** for all the face images stored in the "images" folder. **"image_to_embedding"** function pass an image to the Inception network to generate the embedding vector.

In [24]:
def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding


## About <font color=blue>recognize_face</font> function
This function calculate similarity between the captured image and the images that are already been stored. It passes the image to the trained neural network to generate its embedding vector. Which is then compared with all the embedding vectors of the images stored by calculating L2 Euclidean distance. 

If the minimum L2 distance between two embeddings is less than a threshpld (here I have taken the threashhold as .68 (which can be adjusted) then we have a match.

In [33]:
def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        

        print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))

        
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    
    if minimum_distance < 0.6:
        return str(name)
    else:
        return None

## About <font color=blue>create_input_image_embeddings</font> function
This function generates 128 dimensional image ebeddings of all the images stored in the "images" directory by feed forwarding the images to a trained neural network. It creates a dictionary with key as the name of the face and value as embedding


## About <font color=blue>recognize_faces_in_cam</font> function
This function capture image from the webcam, detect a face in it and crop the image to have a face only, which is then passed to recognize_face function. 

In [26]:
import glob

def create_input_image_embeddings():
    input_embeddings = {}

    for file in glob.glob("images/*"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        image_file = cv2.imread(file, 1)
        input_embeddings[person_name] = image_to_embedding(image_file, model)

    return input_embeddings

def recognize_faces_in_cam(input_embeddings):
    

    cv2.namedWindow("Face Recognizer")
    vc = cv2.VideoCapture(0)
   

    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        height, width, channels = frame.shape

        
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h

           
            
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)
            
            

            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        
        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", img)

        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()

## Capturing the face image
Following code captures 10 face images of the person. They all are stored in **"images"** folder with the name User_1 to User_10. Select a good captured image from the set of 10 images. Rename it with the name of person and delete rest of them. This image will be used for recognizing the the identity of the person using one shot learning.

In [40]:
cam = cv2.VideoCapture(0)

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

count = 0
while(True):
    ret, img = cam.read()
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(img, 1.3, 5)
    for (x,y,w,h) in faces:
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h
        cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
        count += 1
        # Save the captured image into the datasets folder
        cv2.imwrite("images/User_" + str(count) + ".jpg", img[y1:y2,x1:x2])
        cv2.imshow('image', img)
    k = cv2.waitKey(200) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif count >= 10: # Take 30 face sample and stop video
         break
cam.release()
cv2.destroyAllWindows()

# Lets run the face recognizer program :-)

In [38]:
input_embeddings = create_input_image_embeddings()
recognize_faces_in_cam(input_embeddings)


Euclidean distance from Dirk is 1.1578852
Euclidean distance from jape is 0.92300713
Euclidean distance from Jari is 0.9208459
Euclidean distance from Jasper is 0.83055454
Euclidean distance from Kunal is 1.1091973
Euclidean distance from Sumantra is 1.0643592
Euclidean distance from Wanne is 0.7355898
Euclidean distance from Dirk is 1.2077214
Euclidean distance from jape is 0.8750637
Euclidean distance from Jari is 0.8173589
Euclidean distance from Jasper is 0.76510555
Euclidean distance from Kunal is 1.1379424
Euclidean distance from Sumantra is 1.107537
Euclidean distance from Wanne is 0.79288495
Euclidean distance from Dirk is 1.2357417
Euclidean distance from jape is 0.79282385
Euclidean distance from Jari is 0.87673014
Euclidean distance from Jasper is 0.93174994
Euclidean distance from Kunal is 0.92430437
Euclidean distance from Sumantra is 1.1734129
Euclidean distance from Wanne is 0.63545126
Euclidean distance from Dirk is 1.23927
Euclidean distance from jape is 0.69257057
Euc

Euclidean distance from Wanne is 0.9347154
Euclidean distance from Dirk is 1.1450891
Euclidean distance from jape is 0.9890082
Euclidean distance from Jari is 1.0141553
Euclidean distance from Jasper is 0.9413426
Euclidean distance from Kunal is 1.0957946
Euclidean distance from Sumantra is 1.0430607
Euclidean distance from Wanne is 0.96350867
Euclidean distance from Dirk is 1.197309
Euclidean distance from jape is 0.9564606
Euclidean distance from Jari is 0.9934932
Euclidean distance from Jasper is 0.9369049
Euclidean distance from Kunal is 1.1071813
Euclidean distance from Sumantra is 1.0614566
Euclidean distance from Wanne is 0.99687845
Euclidean distance from Dirk is 1.3337033
Euclidean distance from jape is 0.9781859
Euclidean distance from Jari is 0.9743707
Euclidean distance from Jasper is 0.96753824
Euclidean distance from Kunal is 1.098831
Euclidean distance from Sumantra is 0.9640583
Euclidean distance from Wanne is 1.040442
Euclidean distance from Dirk is 1.2894303
Euclidean

Euclidean distance from Dirk is 1.315354
Euclidean distance from jape is 1.1379048
Euclidean distance from Jari is 1.0880282
Euclidean distance from Jasper is 1.0002782
Euclidean distance from Kunal is 1.1370107
Euclidean distance from Sumantra is 0.95939785
Euclidean distance from Wanne is 1.1643035
Euclidean distance from Dirk is 1.3330723
Euclidean distance from jape is 1.1424565
Euclidean distance from Jari is 1.1221212
Euclidean distance from Jasper is 1.0322146
Euclidean distance from Kunal is 1.19511
Euclidean distance from Sumantra is 0.9952565
Euclidean distance from Wanne is 1.203739
Euclidean distance from Dirk is 1.380609
Euclidean distance from jape is 1.1419384
Euclidean distance from Jari is 1.0896654
Euclidean distance from Jasper is 1.052249
Euclidean distance from Kunal is 1.1685117
Euclidean distance from Sumantra is 0.94483167
Euclidean distance from Wanne is 1.2571981
Euclidean distance from Dirk is 1.3083583
Euclidean distance from jape is 1.0187012
Euclidean dist

Euclidean distance from Dirk is 1.2177428
Euclidean distance from jape is 1.0006671
Euclidean distance from Jari is 0.959382
Euclidean distance from Jasper is 0.9189735
Euclidean distance from Kunal is 1.160695
Euclidean distance from Sumantra is 1.1468083
Euclidean distance from Wanne is 1.0415109
Euclidean distance from Dirk is 1.1197908
Euclidean distance from jape is 0.8512929
Euclidean distance from Jari is 0.9429189
Euclidean distance from Jasper is 0.89595747
Euclidean distance from Kunal is 0.94037026
Euclidean distance from Sumantra is 1.0480194
Euclidean distance from Wanne is 0.61050415
Euclidean distance from Dirk is 1.2710958
Euclidean distance from jape is 0.9141335
Euclidean distance from Jari is 0.9571808
Euclidean distance from Jasper is 0.875038
Euclidean distance from Kunal is 1.0500386
Euclidean distance from Sumantra is 1.1488652
Euclidean distance from Wanne is 1.0113403
Euclidean distance from Dirk is 1.2611825
Euclidean distance from jape is 0.68071365
Euclidean

Euclidean distance from Dirk is 1.1517934
Euclidean distance from jape is 0.90984774
Euclidean distance from Jari is 0.90268266
Euclidean distance from Jasper is 0.79578745
Euclidean distance from Kunal is 1.0950332
Euclidean distance from Sumantra is 1.2061385
Euclidean distance from Wanne is 0.9852362
Euclidean distance from Dirk is 1.1861551
Euclidean distance from jape is 0.865371
Euclidean distance from Jari is 0.98939896
Euclidean distance from Jasper is 1.0048914
Euclidean distance from Kunal is 0.97078854
Euclidean distance from Sumantra is 0.9544908
Euclidean distance from Wanne is 0.8257401
Euclidean distance from Dirk is 0.96242946
Euclidean distance from jape is 1.0903856
Euclidean distance from Jari is 1.0797256
Euclidean distance from Jasper is 0.8963465
Euclidean distance from Kunal is 1.2193968
Euclidean distance from Sumantra is 1.20283
Euclidean distance from Wanne is 1.0498049
Euclidean distance from Dirk is 1.2499684
Euclidean distance from jape is 0.846427
Euclidea

...

Euclidean distance from Dirk is 1.3374804
Euclidean distance from jape is 0.5756769
Euclidean distance from Jari is 0.6965563
Euclidean distance from Jasper is 0.77110445
Euclidean distance from Kunal is 0.8259713
Euclidean distance from Sumantra is 1.0633401
Euclidean distance from Wanne is 0.5638725
Euclidean distance from Dirk is 1.4173932
Euclidean distance from jape is 0.8367242
Euclidean distance from Jari is 0.7720154
Euclidean distance from Jasper is 0.9176307
Euclidean distance from Kunal is 1.0770869
Euclidean distance from Sumantra is 0.9537789
Euclidean distance from Wanne is 0.82608736
Euclidean distance from Dirk is 1.416343
Euclidean distance from jape is 0.84957325
Euclidean distance from Jari is 0.729278
Euclidean distance from Jasper is 0.8310063
Euclidean distance from Kunal is 1.0508587
Euclidean distance from Sumantra is 0.9420945
Euclidean distance from Wanne is 0.8809715
Euclidean distance from Dirk is 1.3573246
Euclidean distance from jape is 0.6934249
Euclidean

Euclidean distance from Dirk is 1.2448744
Euclidean distance from jape is 0.5949962
Euclidean distance from Jari is 0.79459184
Euclidean distance from Jasper is 0.8266449
Euclidean distance from Kunal is 0.859688
Euclidean distance from Sumantra is 1.1019723
Euclidean distance from Wanne is 0.6813864
Euclidean distance from Dirk is 1.3250604
Euclidean distance from jape is 0.8114135
Euclidean distance from Jari is 0.7296976
Euclidean distance from Jasper is 0.76718056
Euclidean distance from Kunal is 1.029018
Euclidean distance from Sumantra is 0.9081568
Euclidean distance from Wanne is 0.71472394
Euclidean distance from Dirk is 1.1744438
Euclidean distance from jape is 0.83498955
Euclidean distance from Jari is 0.84886205
Euclidean distance from Jasper is 0.87136596
Euclidean distance from Kunal is 1.0330696
Euclidean distance from Sumantra is 0.9807349
Euclidean distance from Wanne is 0.73856616
Euclidean distance from Dirk is 1.2083396
Euclidean distance from jape is 0.8252786
Eucli

Euclidean distance from Dirk is 1.2740451
Euclidean distance from jape is 0.8779595
Euclidean distance from Jari is 0.8777412
Euclidean distance from Jasper is 0.8357848
Euclidean distance from Kunal is 0.911682
Euclidean distance from Sumantra is 0.9158132
Euclidean distance from Wanne is 0.7306971
Euclidean distance from Dirk is 1.1911973
Euclidean distance from jape is 0.90584224
Euclidean distance from Jari is 0.9411553
Euclidean distance from Jasper is 0.80414975
Euclidean distance from Kunal is 1.084431
Euclidean distance from Sumantra is 0.99926203
Euclidean distance from Wanne is 0.7826224
Euclidean distance from Dirk is 1.0849278
Euclidean distance from jape is 0.90015256
Euclidean distance from Jari is 1.1130662
Euclidean distance from Jasper is 0.88907635
Euclidean distance from Kunal is 0.99507
Euclidean distance from Sumantra is 1.1749926
Euclidean distance from Wanne is 0.5699187
Euclidean distance from Dirk is 1.1615877
Euclidean distance from jape is 0.87133133
Euclidea

Euclidean distance from Dirk is 1.0469726
Euclidean distance from jape is 0.81753594
Euclidean distance from Jari is 1.0519911
Euclidean distance from Jasper is 0.83591545
Euclidean distance from Kunal is 1.0547194
Euclidean distance from Sumantra is 1.3100142
Euclidean distance from Wanne is 0.7202734
Euclidean distance from Dirk is 1.1485378
Euclidean distance from jape is 0.71034425
Euclidean distance from Jari is 0.9025615
Euclidean distance from Jasper is 0.7860831
Euclidean distance from Kunal is 1.0203851
Euclidean distance from Sumantra is 1.2293906
Euclidean distance from Wanne is 0.7195074
Euclidean distance from Dirk is 1.135926
Euclidean distance from jape is 0.7437546
Euclidean distance from Jari is 0.9346227
Euclidean distance from Jasper is 0.8106722
Euclidean distance from Kunal is 1.0204765
Euclidean distance from Sumantra is 1.3026872
Euclidean distance from Wanne is 0.6976714
Euclidean distance from Dirk is 1.191238
Euclidean distance from jape is 0.65979505
Euclidea

Euclidean distance from Dirk is 1.2573763
Euclidean distance from jape is 0.6800906
Euclidean distance from Jari is 0.7755232
Euclidean distance from Jasper is 0.9161441
Euclidean distance from Kunal is 0.9928427
Euclidean distance from Sumantra is 1.1310166
Euclidean distance from Wanne is 0.7589136
Euclidean distance from Dirk is 1.2917733
Euclidean distance from jape is 0.85719395
Euclidean distance from Jari is 0.8956318
Euclidean distance from Jasper is 0.9308461
Euclidean distance from Kunal is 0.96598923
Euclidean distance from Sumantra is 0.9778144
Euclidean distance from Wanne is 0.8726838
Euclidean distance from Dirk is 1.2341492
Euclidean distance from jape is 0.7154143
Euclidean distance from Jari is 0.6824277
Euclidean distance from Jasper is 0.60428894
Euclidean distance from Kunal is 0.9520569
Euclidean distance from Sumantra is 1.1117678
Euclidean distance from Wanne is 0.7638034
Euclidean distance from Dirk is 1.178267
Euclidean distance from jape is 0.849453
Euclidean